In [1]:
import math

from solid import *
from solid.utils import *

from subprocess import run

import viewscad
r = viewscad.Renderer()

In [2]:
import os, solid;
print(os.path.dirname(solid.__file__) + '/examples')

/home/flo/.local/lib/python3.9/site-packages/solid/examples


In [3]:
# CONFIGURATION
GENERATE_WITH_KEYCAPS = False
HOLE_HEIGHT_TOLERANCE = 0.2 # 0 gives weird viewing glitches

In [4]:
KEYCAP_SIZE = 18
KEYCAP_HEIGHT = 8
KEYCAP_Z_OFFSET = 6.6

KNOB_RADIUS = 7
KNOB_HEIGHT = 16
KNOB_Z_OFFSET = 4

PLATE_GRID = 19.05
PLATE_HOLE = 14
PLATE_THICKNESS = 4
# PLATE_SPACER = SWITCH_GRID - PLATE_HOLE  # = 5.05mm

def plate_cutout(units_x=1, units_y=1, withCap=False, encoder=False, center=True):
    if units_x == 0 or units_y == 0:
        withCap = False
        if units_x == 0:
            units_x = 1
            units_y = 1
            cutout = cube([PLATE_GRID,PLATE_GRID,PLATE_THICKNESS], center=True)
        if units_y == 0:
            units_y = 1
            units_x = 1
            cutout = cube(0)
    else:
        plate = cube([PLATE_GRID*units_x,PLATE_GRID*units_y,PLATE_THICKNESS], center=True)
        hole = cube([PLATE_HOLE,PLATE_HOLE,PLATE_THICKNESS+HOLE_HEIGHT_TOLERANCE], center=True)
        if encoder:
            width = 8.37/2 # mm, from the center
            height = PLATE_HOLE/2 + 1.7 # mm, from the center
            filler_points = [
                [-width,height,0],
                [width,height,0],
                [width,-height,0],
                [-width,-height,0],
                [-width,PLATE_HOLE/2,PLATE_THICKNESS*4/5],
                [width,PLATE_HOLE/2,PLATE_THICKNESS*4/5],
                [width,-PLATE_HOLE/2,PLATE_THICKNESS*4/5],
                [-width,-PLATE_HOLE/2,PLATE_THICKNESS*4/5]
            ]
            filler_faces = [
                [3,2,1,0],
                [4,5,6,7],
                [0,1,5,4],
                [1,2,6,5],
                [2,3,7,6],
                [3,0,4,7],
            ]
            encoder_cutout = polyhedron(points=filler_points,faces=filler_faces,convexity=5)
            hole += down(PLATE_THICKNESS/2)(encoder_cutout)
        cutout = plate - hole
    
    # TODO: if units_x or units_y >= 2: add stab holes

    if withCap:
        if encoder:
            knob = color([0.0,1.0,0.5])(cylinder(r=KNOB_RADIUS, h=KNOB_HEIGHT, center=True))
            knob = up(PLATE_THICKNESS/2 + KNOB_HEIGHT/2 + KNOB_Z_OFFSET)(knob)
            cutout += knob
        else:
            spacing = (PLATE_GRID - KEYCAP_SIZE)
            cap = color([0.0,1.0,0.5])(cube([PLATE_GRID*units_x - spacing,PLATE_GRID*units_y - spacing,KEYCAP_HEIGHT], center=True))
            cap = up(PLATE_THICKNESS/2 + KEYCAP_HEIGHT/2 + KEYCAP_Z_OFFSET)(cap)
            cutout += cap
        
    
    if(center == False):
        cutout = left(units_x*PLATE_GRID/2)(back(units_y*PLATE_GRID/2)(cutout))
    
    return cutout

r.render(plate_cutout(units_x=1, units_y=1, withCap=True, encoder=True, center=True))

In [5]:
# keys[units_y], units_x, center, encoder[]
def plate_column(keys=[1,1,1,1], units_x=1, center=True, encoder=[]):
    if len(encoder) == 0 or len(encoder) != len(keys):
        encoder = [0] * len(keys)
    i,offset,column = 0,0,cube(0)
    for units_y in keys:
        cutout = plate_cutout(units_x=units_x, units_y=units_y, withCap=GENERATE_WITH_KEYCAPS, encoder=encoder[i])
        if units_y == 0:
            units_y = 1
        cur_offset = units_y/2
        column += forward(PLATE_GRID*(offset + cur_offset))(cutout)
        offset += (units_y/2) + cur_offset
        i += 1
    if(center == False):
        column = right(units_x*PLATE_GRID/2)(column)
    return column
        
r.render(plate_column(keys=[2,0,2,1], units_x=1.5, center=False, encoder=[0,0,0,1]))

In [6]:
# [column[keys[],units,y_offset_units,encoder[]]]
# keep the y_offset_units in multiple or halfs of 1u -> for easier alignment in KiCAD later

def plate(layout):
    i,cur_offset,offset,plate = 0,0,0,cube(0)
    for column_specs in layout:
        if len(column_specs) == 4:
            encoder = column_specs[3]
        else:
            encoder = []
        column = plate_column(keys=column_specs[0], units_x=column_specs[1], center=True, encoder=encoder)
        cur_offset = column_specs[1]/2
        plate += right(PLATE_GRID*(offset+cur_offset))(forward(PLATE_GRID*column_specs[2])(column))
        offset += column_specs[1]/2 + cur_offset
        i += 1
    return plate
    

r.render(plate([[[2,1,1.5,1],1.5,0],[[1,1,1,1.5],1,0.5],[[1,1,1,1,2],2,-0.5],[[1,2,1],1,0,[1,1,1]]]))

In [7]:
C4 = [1,1,1,1]
C5 = [1,1,1,1,1]
C3_vert = [1.5,1.5,1]
C4_oled = [0,0,0,1]

E5 = [1,0,0,0,0]

def myPlate(layout=[C4,1,0], space_angle_degrees=12, space_units=1.5, thumbcluster_keys=[1,1], ortho=False):
    myPlate = plate(layout)

    space_angle = math.radians(space_angle_degrees) # TODO den nicesten Winkel finden ( ͡° ͜ʖ ͡°)

    # predefined by my layout:
    space_offset_columns = 5

    space_offset_y = layout[space_offset_columns][2] - layout[space_offset_columns-1][2]
    filler_faces = [[0,2,1],[0,1,4,3],[1,2,5,4],[2,0,3,5],[3,4,5]]

    i,plate_x,space_offset_available = 0,0,0 # plate x axis length; amount of units left after the offset
    for column in layout:
        if i >= space_offset_columns:
            space_offset_available += column[1]
        plate_x += column[1]
        i += 1
    
    space = plate_cutout(units_x=space_units, center=False, withCap=GENERATE_WITH_KEYCAPS)
    space = right(PLATE_GRID*space_units)(space) # set anchor on top right edge
    thumbcluster = plate_column(keys=thumbcluster_keys, units_x=1, center=False)

    if(space_angle >= math.pi/4):
        # idk why I calculated that one as well
        adjacent = math.cos(math.pi/2-space_angle)*PLATE_GRID
        opposite = math.sin(math.pi/2-space_angle)*PLATE_GRID
        # this probably doesn't work as expected. I realised too late that this isn't even the angle I wanted
        space_edge_y = opposite + space_units*PLATE_GRID/math.sin(space_angle)
        space_edge_x = (space_edge_y-opposite)/math.tan(space_angle)
    else:
        # this is the working and tested version of this code
        adjacent = space_offset_y*PLATE_GRID/math.tan(math.pi/2-space_angle)
        opposite = space_offset_y*PLATE_GRID
        hypotenuse =  math.hypot(adjacent, opposite)

        opposite_offset = (PLATE_GRID-hypotenuse)/math.cos(space_angle)
        adjacent_offset = (PLATE_GRID-hypotenuse)*math.sin(space_angle)

        space_edge_y = opposite + opposite_offset + space_units*PLATE_GRID*math.sin(space_angle)
        space_edge_x = space_units*PLATE_GRID*math.cos(space_angle) - adjacent_offset


    # position the thumbcluster
    angle_space_thumbcluster = math.atan((space_offset_available*PLATE_GRID - space_edge_x)/space_edge_y) - space_angle
    if angle_space_thumbcluster < 0:
        angle_space_thumbcluster = 0
    
    print("hyp:" + str(hypotenuse))
    print("deg:" + str(math.degrees(angle_space_thumbcluster)))

    mini_spacing = PLATE_GRID*math.tan(angle_space_thumbcluster) # parallel distance from the top right edge of the space key to the thumbcluster
    mini_spacing_hypotenuse = PLATE_GRID/math.cos(angle_space_thumbcluster)
    filler_points = [
        [0,0,-PLATE_THICKNESS/2],
        [mini_spacing,PLATE_GRID,-PLATE_THICKNESS/2],
        [0,PLATE_GRID,-PLATE_THICKNESS/2],
        [0,0,PLATE_THICKNESS/2],
        [mini_spacing,PLATE_GRID,PLATE_THICKNESS/2],
        [0,PLATE_GRID,PLATE_THICKNESS/2]
    ]
    filler = polyhedron(points=filler_points,faces=filler_faces,convexity=5) # between space and thumb keys
    space += back(PLATE_GRID)(right(space_units*PLATE_GRID)(filler))

    thumbcluster = rotate(a=math.degrees(angle_space_thumbcluster), v=DOWN_VEC)(thumbcluster) # rotate
    thumbcluster = back(PLATE_GRID)(right(space_units*PLATE_GRID)(thumbcluster)) # move next to space
    space += thumbcluster

    if ortho:
        pfusch = 1 # mm
        thumbkey = plate_cutout(units_x=1, units_y=1, center=True, withCap=GENERATE_WITH_KEYCAPS, encoder=True) # space is too tight for a cap
        thumbkey = back(PLATE_GRID/2)(left(PLATE_GRID/2)(rotate(a=90, v=DOWN_VEC)(thumbkey)))
        thumbkey = right(mini_spacing + space_units*PLATE_GRID + pfusch)(forward(PLATE_GRID)(thumbkey))
        space += thumbkey
        filler_x = (space_units-1)*PLATE_GRID + mini_spacing + pfusch
        filler_y = math.tan(space_angle)*filler_x
        filler_points = [
            [0,0,-PLATE_THICKNESS/2],
            [filler_x,filler_y,-PLATE_THICKNESS/2],
            [filler_x,0,-PLATE_THICKNESS/2],
            [0,0,PLATE_THICKNESS/2],
            [filler_x,filler_y,PLATE_THICKNESS/2],
            [filler_x,0,PLATE_THICKNESS/2]
        ]
        filler = polyhedron(points=filler_points,faces=filler_faces,convexity=5) # between space, the encoder and the columns above
        space += filler
    
    space = rotate(a=math.degrees(space_angle), v=DOWN_VEC)(space) # rotate
    space_offset_y = layout[space_offset_columns][2]
    space_offset_x = plate_x - space_offset_available
    
    filler_points = [
        [0,0,-PLATE_THICKNESS/2],
        [adjacent,0,-PLATE_THICKNESS/2],
        [0,-opposite,-PLATE_THICKNESS/2],
        [0,0,PLATE_THICKNESS/2],
        [adjacent,0,PLATE_THICKNESS/2],
        [0,-opposite,PLATE_THICKNESS/2]
    ]
    filler = polyhedron(points=filler_points,faces=filler_faces,convexity=5) # between space and column to the left
    myPlate += right(space_offset_x*PLATE_GRID)(forward(space_offset_y*PLATE_GRID)(filler))
    myPlate += right(space_offset_x*PLATE_GRID + adjacent)(forward(space_offset_y*PLATE_GRID)(space))

    filler_x = (space_units*PLATE_GRID+mini_spacing)*math.cos(space_angle) + adjacent
    filler_y = -(space_units*PLATE_GRID+mini_spacing)*math.sin(space_angle)
    mini_spacing2 = math.hypot(filler_x - space_offset_available*PLATE_GRID, filler_y - 0) # length of the opposite of the current filler
    if not ortho:
        filler_points = [
            [adjacent,0,-PLATE_THICKNESS/2],
            [filler_x,filler_y,-PLATE_THICKNESS/2],
            [space_offset_available*PLATE_GRID,0,-PLATE_THICKNESS/2],
            [adjacent,0,PLATE_THICKNESS/2],
            [filler_x,filler_y,PLATE_THICKNESS/2],
            [space_offset_available*PLATE_GRID,0,PLATE_THICKNESS/2]
        ]
        filler = polyhedron(points=filler_points,faces=filler_faces,convexity=5) # between space and the two columns above
        myPlate += right(space_offset_x*PLATE_GRID)(forward(space_offset_y*PLATE_GRID)(filler))

    # get size of thumb cluster
    i,thumbcluster_length = 0,0
    while(i < len(thumbcluster_keys)):
        thumbcluster_length += thumbcluster_keys[i]*PLATE_GRID
        i += 1
    adjacent = thumbcluster_length - mini_spacing2 - mini_spacing_hypotenuse
    filler_x = math.sin(space_angle + angle_space_thumbcluster)*adjacent # adjacent is the hypotenuse in this case
    filler_y = math.cos(space_angle + angle_space_thumbcluster)*adjacent # adjacent is the hypotenuse in this case
    hypotenuse = adjacent/math.cos(space_angle + angle_space_thumbcluster)
    print("hyp:" + str(hypotenuse))
    if not ortho:
        filler_points = [
            [0,0,-PLATE_THICKNESS/2],
            [filler_x,filler_y,-PLATE_THICKNESS/2],
            [0,hypotenuse,-PLATE_THICKNESS/2],
            [0,0,PLATE_THICKNESS/2],
            [filler_x,filler_y,PLATE_THICKNESS/2],
            [0,hypotenuse,PLATE_THICKNESS/2]
        ]
        filler = polyhedron(points=filler_points,faces=filler_faces,convexity=5) # between thumb keys and the grid column to the left
        myPlate += right(plate_x*PLATE_GRID)(forward(space_offset_y*PLATE_GRID)(filler))
    
    return myPlate

r.render(myPlate(
    layout=[[C4,1,0],[C4,1,0],[C5,1,-0.5,E5],[C5,1,-0.375],[C5,1,-0.5],[C4,1,0.25],[C4_oled,1,0.25]],
    space_angle_degrees=12,
    space_units=1.5,
    ortho=True
))

hyp:14.606691249134107
deg:12.085690935967316
hyp:12.20823445316159


In [8]:
from euclid3 import Vector2, Vector3, Point2, Point3

def myCaseTop(layout, ortho=False):
    ä = 2
    points = [
        Point2(-ä,-ä),
        Point2(PLATE_GRID,-ä),
        Point2(2*PLATE_GRID-ä,-0.5*PLATE_GRID-ä),
        Point2(140-ä,-26-ä)
    ]
    # In its simplest form, catmull_rom_polygon() will just make a C1-continuous
    # closed shape. Easy.
    shape_easy = catmull_rom_polygon(points)
    # There are some other options as well...
#     shape = catmull_rom_polygon(points, subdivisions=20, extrude_height=5, show_controls=True)
    return shape_easy #+ right(3)(shape)
    
#     plate = myPlate(layout=layout, space_angle_degrees=12, space_units=1.5, ortho=ortho)
#     case = cube()
#     case += plate
#     return case

r.render(myCaseTop(
    layout=[[C4,1.5,0],[C4,1,0],[C5,1,-0.5,E5],[C5,1,-0.375],[C5,1,-0.5],[C4,1,0.25],[C4,1,0.25]],
    ortho=False
))

NameError: name 'catmull_rom_polygon' is not defined

In [ ]:
layouts = [
    [[C4,1.5,0],[C4,1,0],[C5,1,-0.5,E5],[C5,1,-0.25],[C5,1,-0.5],[C4,1,0.25],[C3_vert,1,0.25]], # ergo caps
    [[C4,1,0],[C4,1,0],[C5,1,-0.5,E5],[C5,1,-0.25],[C5,1,-0.5],[C4,1,0.25],[C4_oled,1,0.25]] # ortho caps (+OLED)
]
angle = 12
space_keys = [
    1.5, # normal sizing
#     1.75 # 40% sizing --- CANCELLED, not worth it ---
]
orthos = [
    False,
    True
]

i = 0
while i < len(layouts):
    mp = myPlate(layout=layouts[i],space_angle_degrees=12,space_units=space_keys[0],ortho=orthos[i])
    r.render(mp)
    getSVG(mp, "layout"+str(i))
    i += 1


# r.render(myPlate(layout=layouts[0],space_angle_degrees=12,space_units=space_keys[0]))
# scad_render_to_file(myPlate(layout=layouts[0],space_angle_degrees=12,space_units=space_keys[0]), 'plate_0.scad')
# scad_render_to_file(myPlate(layout=layouts[0],space_angle_degrees=12,space_units=space_keys[1]), 'plate_1.scad')
# r.render(myPlate(layout=layouts[1],space_angle_degrees=12,space_units=space_keys[0],ortho=True))

In [ ]:
def getSVG(object, name="projection"):
    scad_render_to_file(projection(cut=True)(object), name + ".scad")
    run(["openscad", "-o",  name + ".svg", name + ".scad"])

# getSVG(myP, "projection2")

In [ ]:
# generate valid openscad code and store it in file
scad_render_to_file(myPlate(layout=layouts[0],space_angle_degrees=12,space_units=space_keys[0]), 'plate.scad')

# run openscad and export to stl
run(["openscad", "-o",  "plate.stl", "plate.scad"])

In [ ]:
#! /usr/bin/env python
import os
import sys
from solid import *
from solid.utils import Red, right, forward, back

from solid.splines import catmull_rom_points, catmull_rom_polygon, control_points
from solid.splines import bezier_polygon, bezier_points
from euclid3 import Vector2, Vector3, Point2, Point3

def assembly():
    # Catmull-Rom Splines
    a = basic_catmull_rom()                         # Top row in OpenSCAD output
    a += back(4)(catmull_rom_spline_variants())     # Row 2
    a += back(12)(bottle_shape(width=2, height=6))  # Row 3, the bottle shape

    # # TODO: include examples for 3D surfaces:
    # a += back(16)(catmull_rom_patches())
    # a += back(20)(catmull_rom_prism())
    # a += back(24)(catmull_rom_prism_smooth())

    # Bezier Splines
    a += back(16)(basic_bezier())                   # Row 4
    a += back(20)(bezier_points_variants())         # Row 5
    return a

def basic_catmull_rom():
    points = [
        Point2(0,0),
        Point2(1,1),
        Point2(2,1),
        Point2(2,-1),
    ]
    # In its simplest form, catmull_rom_polygon() will just make a C1-continuous
    # closed shape. Easy.
    shape_easy = catmull_rom_polygon(points)
    # There are some other options as well...
    shape = catmull_rom_polygon(points, subdivisions=20, extrude_height=5, show_controls=True)
    return shape_easy + right(3)(shape)

def catmull_rom_spline_variants():
    points = [
        Point2(0,0),
        Point2(1,1),
        Point2(2,1),
        Point2(2,-1),
    ]
    controls = control_points(points)

    # By default, catmull_rom_points() will return a closed smooth shape
    curve_points_closed = catmull_rom_points(points, close_loop=True)

    # If `close_loop` is False, it will return only points between the start and
    # end control points, and make a best guess about tangents for the first and last segments
    curve_points_open   = catmull_rom_points(points, close_loop=False)
    
    # By specifying start_tangent and end_tangent, you can change a shape 
    # significantly. This is similar to what you might do with Illustrator's Pen Tool.
    # Try changing these vectors to see the effects this has on the rightmost curve in the example
    start_tangent = Vector2(-2, 0)
    end_tangent = Vector2(3, 0)
    tangent_pts = [points[0] + start_tangent, *points, points[-1] + end_tangent]
    tangent_controls = control_points(tangent_pts)
    curve_points_tangents = catmull_rom_points(points, close_loop=False, 
                                start_tangent=start_tangent, end_tangent=end_tangent)

    closed = polygon(curve_points_closed) + controls
    opened = polygon(curve_points_open) + controls
    tangents = polygon(curve_points_tangents) + tangent_controls

    a = closed + right(3)(opened) + right(10)(tangents)

    return a

def catmull_rom_patches():
    # TODO: write this
    pass

def catmull_rom_prism():
    # TODO: write this
    pass

def catmull_rom_prism_smooth():
    # TODO: write this
    pass

def bottle_shape(width: float, height: float, neck_width:float=None, neck_height:float=None):
    if neck_width == None:
        neck_width = width * 0.4
    
    if neck_height == None:
        neck_height = height * 0.2

    w2 = width/2
    nw2 = neck_width/2
    h = height
    nh = neck_height

    corner_rad = 0.5

    # Add extra tangent points near curves to keep cubics from going crazy. 
    # Try taking some of these out and see how this affects the final shape
    points = [
        Point2(nw2, h),
        Point2(nw2, h-nh + 1),      # <- extra tangent
        Point2(nw2, h - nh),    
        Point2(w2, h-nh-h/6),       # <- extra tangent
        Point2(w2, corner_rad + 1), # <- extra tangent
        Point2(w2, corner_rad),
        Point2(w2-corner_rad, 0),
        Point2(0,0),
    ]
    # Use catmull_rom_points() when you don't want all corners in a polygon 
    # smoothed out or want to combine the curve with other shapes. 
    # Extra points can then be added to the list you get back
    cr_points = catmull_rom_points(points)

    # Insert a point at the top center of the bottle at the beginning of the 
    # points list. This is how the bottle has a sharp right angle corner at the 
    # sides of the neck; otherwise we'd have to insert several extra control 
    # points to make a sharp corner
    cr_points.insert(0, (0,h))
    
    # Make OpenSCAD polygons out of the shapes once all points are calculated
    a = polygon(cr_points) 
    a += mirror(v=(1,0))(a)

    # Show control points. These aren't required for anything, but seeing them
    # makes refining a curve much easier
    controls = control_points(points)
    a += controls
    return a

def basic_bezier():
    # A basic cubic Bezier curve will pass through its first and last 
    # points, but not through the central control points
    controls = [
        Point2(0, 3),
        Point2(1, 1),
        Point2(2, 1),
        Point2(3, 3)
    ]
    shape = bezier_polygon(controls, show_controls=True)
    return shape

def bezier_points_variants():
    controls = [
        Point2(0,0),
        Point2(1,2),
        Point2(2, -1),
        Point2(3,0),
    ]
    points = bezier_points(controls, subdivisions=20)
    # For non-smooth curves, add extra points
    points += [
        Point2(2, -2),
        Point2(1, -2)
    ]
    shape = polygon(points) + control_points(controls, extrude_height=0)
    return shape


out_dir = "/home/flo/Repositories/keyboards/curly-enigma/case/-f"

a = assembly()

out_path = scad_render_to_file(a, out_dir=out_dir, include_orig_code=True)

I should have probably gone with a vectore based approach…